<a href="https://colab.research.google.com/github/saythegreat/Machine-Learning/blob/main/ML_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Lab08 - Neural Networks and Perceptrons
Clean solution (A1–A12), structured by question.
"""

import numpy as np
import pandas as pd
from numpy.linalg import pinv
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# ================= A1: Summation Unit =================
def summation_unit(x, w):
    return float(np.dot(w, x))

# ================= A2: Activation Functions =================
def step_activation(x): return np.where(x >= 0, 1, 0)
def bipolar_step_activation(x): return np.where(x >= 0, 1, -1)
def sigmoid_activation(x): return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
def tanh_activation(x): return np.tanh(x)
def relu_activation(x): return np.maximum(0, x)
def leaky_relu_activation(x, alpha=0.01): return np.where(x > 0, x, alpha * x)

# ================= A3: Comparator / Error Unit =================
def sse_error(y_true, y_pred):
    return float(0.5 * np.sum((y_true - y_pred) ** 2))

# ================= A4: Perceptron Training =================
def perceptron_train(X, y, initial_w, lr=0.05, activation="step",
                     max_epochs=1000, convergence_sse=0.002):
    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float)
    n = X.shape[0]
    Xb = np.hstack([np.ones((n, 1)), X])
    w = initial_w.astype(float).copy()
    sse_history = []
    converged = False

    for epoch in range(1, max_epochs + 1):
        net = Xb @ w
        if activation == "step":
            out = step_activation(net)
            deltas = (y - out)
            w += lr * (deltas @ Xb)
        elif activation == "bipolar":
            y_b = np.where(y == 0, -1, 1)
            out = bipolar_step_activation(net)
            deltas = (y_b - out)
            w += lr * (deltas @ Xb)
        elif activation == "sigmoid":
            out = sigmoid_activation(net)
            deltas = (y - out) * out * (1 - out)
            w += lr * (deltas @ Xb)
        elif activation == "relu":
            out = step_activation(net)
            deltas = (y - out)
            w += lr * (deltas @ Xb)

        sse = sse_error(y, out)
        sse_history.append(round(sse, 4))
        if sse <= convergence_sse:
            converged = True
            break

    return {
        "epochs": int(epoch),
        "final_sse": round(float(sse), 4),
        "converged": bool(converged),
        "weights": [round(float(val), 4) for val in w],
        "sse_history": sse_history[:15] + (["..."] if len(sse_history) > 15 else [])
    }

# ================= A5: Datasets =================
def and_dataset():
    return np.array([[0,0],[0,1],[1,0],[1,1]]), np.array([0,0,0,1])
def xor_dataset():
    return np.array([[0,0],[0,1],[1,0],[1,1]]), np.array([0,1,1,0])

# ================= A6: Customer Dataset =================
def customer_dataset():
    data = [
        (20,6,2,386,1),(16,3,6,289,1),(27,6,2,393,1),
        (19,1,2,110,0),(24,4,2,280,1),(22,1,5,167,0),
        (15,4,2,271,1),(18,4,2,274,1),(21,1,4,148,0),(16,2,4,198,0),
    ]
    df = pd.DataFrame(data, columns=["candies","mangoes","milk","payment","high"])
    X = df[["candies","mangoes","milk","payment"]].values.astype(float)
    y = df["high"].values.astype(int)
    return X, y

# ================= A7: Pseudo-Inverse =================
def pseudo_inverse_solution(X, y):
    n = X.shape[0]
    Xb = np.hstack([np.ones((n,1)), X])
    w = pinv(Xb) @ y
    raw = Xb @ w
    preds = (raw >= 0.5).astype(int)
    return {
        "weights": [round(float(val), 4) for val in w],
        "accuracy": float((preds == y).mean())
    }

# ================= A8: Backprop MLP for AND =================
class SmallMLP:
    def __init__(self, n_inputs, n_hidden=2, lr=0.05, max_epochs=1000, conv_sse=0.002):
        rng = np.random.RandomState(0)
        self.W1 = rng.randn(n_hidden, n_inputs+1) * 0.1
        self.W2 = rng.randn(1, n_hidden+1) * 0.1
        self.lr, self.max_epochs, self.conv_sse = lr, max_epochs, conv_sse

    def sigmoid(self,x): return 1/(1+np.exp(-np.clip(x,-500,500)))
    def sigmoid_deriv(self,y): return y*(1-y)

    def forward(self,X):
        n = X.shape[0]
        Xb = np.hstack([np.ones((n,1)),X])
        z1 = self.sigmoid(Xb @ self.W1.T)
        z1b = np.hstack([np.ones((n,1)),z1])
        out = self.sigmoid(z1b @ self.W2.T).ravel()
        return out,Xb,z1,z1b

    def train(self,X,y):
        sse_history=[]
        for epoch in range(1,self.max_epochs+1):
            out,Xb,z1,z1b = self.forward(X)
            err = y-out
            sse=0.5*np.sum(err**2)
            sse_history.append(round(float(sse),4))
            if sse<=self.conv_sse:
                return {"epochs":epoch,"converged":True,"sse_history":sse_history[:15]}
            delta_out = err*self.sigmoid_deriv(out)
            grad_W2 = delta_out.reshape(-1,1).T @ z1b
            delta_hidden = (delta_out.reshape(-1,1) @ self.W2[:,1:]) * self.sigmoid_deriv(z1)
            grad_W1 = delta_hidden.T @ Xb
            self.W2 += self.lr*grad_W2
            self.W1 += self.lr*grad_W1
        return {"epochs":self.max_epochs,"converged":False,"sse_history":sse_history[:15]}

# ================= A11 & A12: sklearn MLP =================
def sklearn_mlp_and_xor():
    X_and,y_and = and_dataset()
    X_xor,y_xor = xor_dataset()
    clf_and = MLPClassifier(hidden_layer_sizes=(),activation="logistic",solver="lbfgs",max_iter=1000)
    clf_and.fit(X_and,y_and)
    clf_xor = MLPClassifier(hidden_layer_sizes=(2,),activation="logistic",solver="lbfgs",max_iter=10000)
    clf_xor.fit(X_xor,y_xor)
    return {
        "AND_acc": float(clf_and.score(X_and,y_and)),
        "XOR_acc": float(clf_xor.score(X_xor,y_xor))
    }

def sklearn_mlp_customer():
    X,y = customer_dataset()
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    clf = MLPClassifier(hidden_layer_sizes=(5,),activation="logistic",max_iter=2000,random_state=0)
    clf.fit(Xs,y)
    preds = clf.predict(Xs)
    return {"accuracy": float((preds==y).mean())}

# ================= MAIN =================
if __name__ == "__main__":
    init_w = np.array([10,0.2,-0.75])
    X_and,y_and = and_dataset()
    X_xor,y_xor = xor_dataset()

    print("A5 AND:", perceptron_train(X_and, y_and, init_w, activation="step"))
    print("A5 XOR:", perceptron_train(X_xor, y_xor, init_w, activation="step"))
    print("A7 Customer pseudo-inverse:", pseudo_inverse_solution(*customer_dataset()))
    mlp = SmallMLP(2)
    print("A8 SmallMLP AND:", mlp.train(X_and,y_and))
    print("A11 sklearn AND/XOR:", sklearn_mlp_and_xor())
    print("A12 sklearn Customer:", sklearn_mlp_customer())


A5 AND: {'epochs': 130, 'final_sse': 0.0, 'converged': True, 'weights': [-0.05, 0.05, 0.05], 'sse_history': [1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, '...']}
A5 XOR: {'epochs': 1000, 'final_sse': 1.0, 'converged': False, 'weights': [-0.05, -0.05, -0.05], 'sse_history': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, '...']}
A7 Customer pseudo-inverse: {'weights': [0.114, -0.0279, 0.0147, -0.0432, 0.0045], 'accuracy': 1.0}
A8 SmallMLP AND: {'epochs': 1000, 'converged': False, 'sse_history': [0.5211, 0.5153, 0.5097, 0.5044, 0.4992, 0.4943, 0.4896, 0.4851, 0.4807, 0.4766, 0.4726, 0.4688, 0.4651, 0.4616, 0.4583]}
A11 sklearn AND/XOR: {'AND_acc': 1.0, 'XOR_acc': 1.0}
A12 sklearn Customer: {'accuracy': 1.0}
